

## Scenario Explanation

While no fraud prevention measures can ever be perfect, significant opportunity for improvement lies in looking beyond the individual data points, to the connections that link them. Oftentimes these connections go unnoticed until it is too late— something that is unfortunate, as these connections oftentimes hold the best clues.

While the exact details behind each first-party fraud collusion vary from operation to operation, the pattern below illustrates how fraud rings commonly operate:
*	A group of two or more people organize into a fraud ring
*	The ring shares a subset of legitimate contact information, for example phone numbers and addresses, combining them to create a number of synthetic identities
*	Ring members open accounts using these synthetic identities
*	New accounts are added to the original ones: unsecured credit lines, credit cards, overdraft protection, personal loans, etc.
*	The accounts are used as normally, with regular purchases and timely payments
*	Banks increase the revolving credit lines over time, due to the observed responsible credit behavior
*	One day the ring “busts out”, coordinating their activity, maxing out all of their credit lines, and disappearing
*	Sometimes fraudsters will go a step further and bring all of their balances to zero using fake checks immediately before the prior step, doubling the damage
*	Collections processes ensue, but agents are never able to reach the fraudster
*	The uncollectible debt is written off


## First Step: Get a session in PGX server.


#### PGX Server Design

 * PGX is designed for a Server-Client usage model. That is, PGX is designed for the following situation: The PGX server is initiated on a server-class machine, while PGX clients (remotely) connect to the PGX server; multiple clients can connect to the same PGX server at the same time.
 * Each PGX client sends out its requests to the PGX server, and the server returns responses for them. 
 * Internally, PGX maintains its own engine (i.e. thread pools) for running parallel graph algorithms. 

![atl text](http://localhost/pgx_multiclient.png "pgx client")

PGX server requires authentication and TLS connection. 

Authentication is managed by the Oracle Database, so the username and password are valid to connect to the database and once authenticated, the session is able to read all objects the user has privileges on.

By default it uses https://server:7007 as entry point.

The paragraph below gets a token valid for 3600 seconds and then creates a client session in PGX that will be used by the Python session.

In [1]:
import json
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

def generateToken(base_url, username, password):
    from urllib.request import Request, urlopen
    from urllib.error import HTTPError

    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)

    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')

    except HTTPError as err:

        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))

        os._exit(1)

baseUrl = 'https://localhost:7007'
username = 'bankuser'
password = 'welcome1'

token=generateToken(baseUrl, username, password)

Now, with the token, some objects are built:
 * *session* this is the first
 * *instance* created from the session
 * *analyst* this one is used for the algorithms

In [2]:
try:
    import pypgx as pgx

    session = pgx.get_session(base_url=baseUrl, token=token)
    instance = session.server_instance
    analyst = session.create_analyst()

except SystemExit:
    os._exit(0)
    

Let's print the session ID. This can be used to log in to Graph Viz so that the graphs can be visualized.

In [3]:
print(session)

PgxSession(id: 3e61915f-b8a5-44ad-8bc1-da2dfabe7aa8, name: pypgx)


Now, all is ready to load the graph from database tables to PGX memory structures.
A json file containing the metadata and mappings from the database table is used for this step.

Let's have a look at the contents of /home/oracle/graphdata/bank_pgviews.json:

```json
{
    "name": "bankpv",
    "vertex_id_strategy": "keys_as_ids",
    "vertex_id_type": "long",
    "vertex_providers": [
        {
            "name": "ACCOUNT",
            "format": "rdbms",
            "database_table_name": "PACCOUNTS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "ACCOUNTNUMBER",
                    "type": "string"
                },
                {
                    "name": "BALANCE",
                    "type": "long"
                }
            ]
        },
        {
            "name": "CUSTOMER",
            "format": "rdbms",
            "database_table_name": "PCUSTOMERS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "FIRSTNAME",
                    "type": "string"
                },
                {
                    "name": "LASTNAME",
                    "type": "string"
                }
            ]
        },
        {
            "name": "CREDITCARD",
            "format": "rdbms",
            "database_table_name": "PCREDITCARDS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "ACCOUNTNUMBER",
                    "type": "string"
                },
                {
                    "name": "LIMIT",
                    "type": "long"
                },
                {
                    "name": "BALANCE",
                    "type": "long"
                },
                {
                    "name": "EXPIRATIONDATE",
                    "type": "timestamp"
                }
            ]
        },
        {
            "name": "UNSECUREDLOAN",
            "format": "rdbms",
            "database_table_name": "PUNSECUREDLOANS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "ACCOUNTNUMBER",
                    "type": "string"
                },
                {
                    "name": "LOANAMOUNT",
                    "type": "long"
                },
                {
                    "name": "BALANCE",
                    "type": "long"
                }
            ]
        },
        {
            "name": "PHONENUMBER",
            "format": "rdbms",
            "database_table_name": "PPHONENUMBERS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "PHONENUMBER",
                    "type": "string"
                }
            ]
        },
        {
            "name": "SSN",
            "format": "rdbms",
            "database_table_name": "PSSNS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "SSN",
                    "type": "string"
                }
            ]
        },
        {
            "name": "ADDRESS",
            "format": "rdbms",
            "database_table_name": "PADDRESSES",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "STREET",
                    "type": "string"
                },
                {
                    "name": "CITY",
                    "type": "string"
                },
                {
                    "name": "STATE",
                    "type": "string"
                },
                {
                    "name": "ZIPCODE",
                    "type": "long"
                },
                {
                    "name": "LATITUDE",
                    "type": "string"
                },
                {
                    "name": "LONGITUDE",
                    "type": "string"
                }
            ]
        }
    ],
    "edge_providers": [
         {
            "name": "HAS_BANKACCOUNT",
            "format": "rdbms",
            "database_table_name": "E_HAS_ACCOUNT",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOACCOUNT",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "ACCOUNT",
            "props": [
               {
                    "name": "CREATIONDATE",
                    "type": "timestamp"
                }
            ]
        },
        {
            "name": "HAS_CREDITCARDS",
            "format": "rdbms",
            "database_table_name": "E_HAS_CREDITCARD",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOCREDITCARD",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "CREDITCARD",
            "props": []
        },
        {
            "name": "HAS_UNSECUREDLOANS",
            "format": "rdbms",
            "database_table_name": "E_HAS_UNSECUREDLOAN",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOUNSECUREDLOAN",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "UNSECUREDLOAN",
            "props": []
        },
        {
            "name": "HAS_PHONENUMBER",
            "format": "rdbms",
            "database_table_name": "E_HAS_PHONENUMBER",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOPHONE",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "PHONENUMBER",
            "props": []
        },
        {
            "name": "HAS_SSN",
            "format": "rdbms",
            "database_table_name": "E_HAS_SSN",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOSSN",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "SSN",
            "props": []
        },
        {
            "name": "HAS_ADDRESS",
            "format": "rdbms",
            "database_table_name": "E_HAS_ADDRESS",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOADDRESS",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "ADDRESS",
            "props": []
        }
    ]
}

```

In [4]:
# read Graph

graph = session.read_graph_with_properties("/home/oracle/graphdata/bank_pgviews.json")


In [ ]:
graph.

Let's have a look at the number of nodes and edges for this graph.

In [5]:
print('Edges number: ' + str(graph.num_edges) + ' Nodes number: '+ str(graph.num_vertices))

Edges number: 347 Nodes number: 535


This is the database model:

![atl text](http://localhost/bankuser_datamodel.png "bankuser schema")

Let's inpect vertex and edges properties.

These are specified in the json file **/home/oracle/graphdata/bank_pgviews.json** so that not all table columns are available in the property graph.

In [6]:
graph.get_vertex_properties()

[VertexProperty(name: ACCOUNTNUMBER, type: string, graph: bankpv),
 VertexProperty(name: BALANCE, type: long, graph: bankpv),
 VertexProperty(name: CITY, type: string, graph: bankpv),
 VertexProperty(name: EXPIRATIONDATE, type: timestamp, graph: bankpv),
 VertexProperty(name: FIRSTNAME, type: string, graph: bankpv),
 VertexProperty(name: LASTNAME, type: string, graph: bankpv),
 VertexProperty(name: LATITUDE, type: string, graph: bankpv),
 VertexProperty(name: LIMIT, type: long, graph: bankpv),
 VertexProperty(name: LOANAMOUNT, type: long, graph: bankpv),
 VertexProperty(name: LONGITUDE, type: string, graph: bankpv),
 VertexProperty(name: PHONENUMBER, type: string, graph: bankpv),
 VertexProperty(name: SSN, type: string, graph: bankpv),
 VertexProperty(name: STATE, type: string, graph: bankpv),
 VertexProperty(name: STREET, type: string, graph: bankpv),
 VertexProperty(name: ZIPCODE, type: long, graph: bankpv)]

In [7]:
graph.get_edge_properties()

[EdgeProperty(name: CREATIONDATE, type: timestamp, graph: bankpv)]

## Entity Link Analysis

Performing entity link analysis on the above Graph data model is demonstrated below. We use brackets in the below table is to isolate individual elements of a collection.

Find accounts who share more than one piece of legitimate contact. These account could be part of a **Fraud Ring**:

A Fraud Ring could be described as an organization which performs activities with the intention to defraud or take advantage of other people. This organization might be involved in any kind of forgery. Actions can range from creating fake claims, stealing a private identity, or even counterfeiting checks and currency. Some rings are devoted to committing fraud against ecommerce websites. Others are devoted to defrauding charities, businesses or government agencies. These organizations can consist of 10 criminals or 10,000. Most are devoted to committing specific types of fraud. [Source](https://fraud.net/d/fraud-ring/)

[PGQL](https://pgql-lang.org/spec/1.3/) language will be used to find vertices of type CUSTOMER who share pieces of legitimate contact.

A subquery is used to find those customers, and then an aggregated list with their names is built.

In [8]:
pgxResultSetNode = graph.query_pgql("""
select label(contact), listagg(acct.FIRSTNAME,','), count(*) as size
from match (acct)-[]->(contact)
where exists (
  SELECT count(*) as ringsize, contact
  from MATCH (account:CUSTOMER)-[]->(contact)
  group by contact 
  having ringsize > 1 
  order by ringsize desc
)
group by label(contact)
""")

for i in pgxResultSetNode:
    print (i)

['ADDRESS', 'Juan,Sonia,Mateo', 3]
['PHONENUMBER', 'Juan,Sonia', 2]
['SSN', 'Sonia,Mateo', 2]


Determine the financial risk of a possible fraud ring. 

At this moment we focus on the products owned directly by them. These customers have creditcards and loans that could be at risk.

This PGQL query looks for those products and their statuses to find out how much money could be at risk. List aggregation and case conditionals are used.

In [9]:
pgxResultSetNode = graph.query_pgql("""
select label(contact), listagg(acct.FIRSTNAME,','), listagg(id(acct),','),
        count(*) as size, round(sum(CASE 
                                     WHEN label(r)='HAS_CREDITCARDS' THEN unsecuredAccount.LIMIT 
                                     WHEN label(r)='HAS_UNSECUREDLOANS' THEN unsecuredAccount.BALANCE 
                                     ELSE 0 
                                    END)) as FINANCIALRISK
from match (acct)-[]->(contact), 
     match (acct)-[r:HAS_CREDITCARDS|HAS_UNSECUREDLOANS]->(unsecuredAccount)
where exists (
  SELECT count(*) as ringsize, contact
  from MATCH (account:CUSTOMER)-[]->(contact)
  group by contact 
  having ringsize > 1 
  order by ringsize desc
)
group by label(contact)
""")

fin_risk = []

for i in pgxResultSetNode:
    print (i)
    fin_risk.append(i)


['ADDRESS', 'Sonia,Mateo,Juan,Sonia', '101,102,100,101', 4, 34386]
['PHONENUMBER', 'Sonia,Juan,Sonia', '101,100,101', 3, 18045]
['SSN', 'Sonia,Mateo,Sonia', '101,102,101', 3, 29386]


Let's create a Pandas data frame to store this information. [Pandas](https://pandas.pydata.org/) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

Also descriptive column names are added.

In [10]:
import pandas as pd

df= pd.DataFrame(fin_risk)
df=df.reset_index(drop=True)
df.rename(columns = {0:'CONTACT_TYPE', 1:'NAMES',2:'CUSTOMERS',3:'RINGSIZE',4:'AMOUNT'}, inplace = True)
df

CONTACT_TYPE                   NAMES        CUSTOMERS  RINGSIZE  AMOUNT
0      ADDRESS  Sonia,Mateo,Juan,Sonia  101,102,100,101         4   34386
1  PHONENUMBER        Sonia,Juan,Sonia      101,100,101         3   18045
2          SSN       Sonia,Mateo,Sonia      101,102,101         3   29386

## Store results back to database

Now, let's use **OML4Py** to store the datafrate as a regular database table so that this information can be used by any application.

No need to pre-create the table or write SQL for this task.

In [11]:
import oml
# Connect with existing oracle database
oml.connect("bankuser","welcome1",
            dsn='(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(service_name=ORCLPDB)))')
# Check whether connection attempt was successful
oml.isconnected()
  
# Save analysis results as table
try:
    oml.drop(table="RING_FRAUD")
except:
    pass
oml.create(df, table = 'RING_FRAUD')

# Close connection
oml.disconnect()

## Graphviz

Let's now visualize the graph using the built-in tool: GraphViz.

Open https://localhost:7007/ui

To log in use the same credentials used in the top of this notebook:
 * user: bankuser
 * password: welcome1
 * session ID

The session ID is needed because graphs created in this session are only visible to the current session. Session ID can be found by inspecting the session object as shown in the paragraph below.

It's a string that looks like this (execute next paragraph to get it): 

```
xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxx
```

In [12]:
print(session)

PgxSession(id: 3e61915f-b8a5-44ad-8bc1-da2dfabe7aa8, name: pypgx)


After logging in to GraphViz, the graph 'bankpv' is ready to execute this PGQL query on it and show the results.

```SQL
select acct,e,contact,o, other
from match (acct)-[e]->(contact) on "bankpv",
     match (acct)-[o]->(other) on "bankpv"
where exists (
  SELECT count(*) as ringsize, contact
  from MATCH (account:CUSTOMER)-[x]->(contact) on "bankpv"
  group by contact 
  having ringsize > 1 
  order by ringsize desc
)
```

Copy and paste the PGQL query, then click the blue play button, to execute the PGQL query and then, on the right side under 'Settings' click on the button with an up arrow to 'Load settings'.


![atl text](http://localhost/gviz-settings1.png "graphviz")

A file selector screen will show up. Look for a file named 'bankpv_settings.json' in the 'graphdata' folder.

![atl text](http://localhost/gviz-settings2.png "graphviz")

Finally you should see something similar to the picture below. 
Drag the vertices to position them, right click to get their properties.

![atl text](http://localhost/bankgraph.png "graphviz")



When finished, destroy graph and close session to free resources in PGX engine.

In [ ]:
graph.destroy()
session.destroy()